<a href="https://colab.research.google.com/github/lasiadhi/Prediction-of-hypoxemia-trend/blob/master/construct_IV_cohort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#################################################
###### @author: Lasith Adhikari (python 3)  #####
############ Created on Nov 25, 2019 ############
# Code description: 
# Construct mechanical ventilation start and stop time(s) dictionary 

In [0]:
# Import libraries
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.path as path
import tensorflow as tf

# Below imports are used to print out pretty pandas dataframes
from IPython.display import display, HTML

# Imports for accessing Datathon data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

In [0]:
from google.colab import files
import seaborn as sns
import pickle
import matplotlib.pyplot as plt 

In [0]:
auth.authenticate_user()

In [0]:
# Note that this should be the project for the datathon work,
# not the physionet-data project which is for data hosting.
project_id = 'hst-953-2019'
os.environ['GOOGLE_CLOUD_PROJECT'] = project_id

In [0]:

# Read data from BigQuery into pandas dataframes.
def run_query(query):
  return pd.io.gbq.read_gbq(
      query,
      project_id=project_id,
      dialect='standard')

# Read dict with IV start and D/C times

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Read dict from mech_event:

In [0]:
vent_event_dict = pickle.load(open("/content/drive/My Drive/Colab Notebooks/data/MVdict_from_ventEventtbl_with_DC.pkl", "rb" ) )


In [12]:
vent_event_dict

{534589: {'ICU Admit': [0.0],
  'ICU Discharge': [7.1],
  'IV': [],
  'NIV': [],
  'O2': []},
 534590: {'ICU Admit': [0.0],
  'ICU Discharge': [22.7],
  'IV': [],
  'NIV': [],
  'O2': []},
 534605: {'ICU Admit': [0.0],
  'ICU Discharge': [46.75],
  'IV': [(-0.083, 14.633)],
  'NIV': [],
  'O2': [(14.633, nan)]},
 534621: {'ICU Admit': [0.0],
  'ICU Discharge': [23.316666666666666],
  'IV': [(-0.317, 4.35)],
  'NIV': [],
  'O2': [(4.35, nan)]},
 534622: {'ICU Admit': [0.0],
  'ICU Discharge': [28.55],
  'IV': [],
  'NIV': [],
  'O2': [(0.283, 16.283), (17.283, nan)]},
 534637: {'ICU Admit': [0.0],
  'ICU Discharge': [18.916666666666668],
  'IV': [],
  'NIV': [],
  'O2': []},
 534930: {'ICU Admit': [0.0],
  'ICU Discharge': [45.35],
  'IV': [],
  'NIV': [(0.5, nan)],
  'O2': [(0.0, nan)]},
 534942: {'ICU Admit': [0.0],
  'ICU Discharge': [26.083333333333332],
  'IV': [],
  'NIV': [],
  'O2': [(-0.667, nan)]},
 534967: {'ICU Admit': [0.0],
  'ICU Discharge': [52.31666666666667],
  'IV': [

In [14]:
len(vent_event_dict.keys())

22859

In [0]:
# Read dict from care plan:

In [0]:
care_plan_dict = pickle.load(open("/content/drive/My Drive/Colab Notebooks/data/MVdict_from_careplane_with_DC.pkl", "rb" ) )


In [13]:
care_plan_dict

{141168: {'ICU Admit': [0], 'ICU Discharge': [59.93], 'IV': [(36.0, nan)]},
 141244: {'ICU Admit': [0], 'ICU Discharge': [63.92], 'IV': []},
 141296: {'ICU Admit': [0], 'ICU Discharge': [47.13], 'IV': []},
 141313: {'ICU Admit': [0], 'ICU Discharge': [7.22], 'IV': [(0.62, nan)]},
 141451: {'ICU Admit': [0], 'ICU Discharge': [64.72], 'IV': [(0.87, 16.45)]},
 141515: {'ICU Admit': [0], 'ICU Discharge': [148.03], 'IV': [(0.27, 148.03)]},
 141585: {'ICU Admit': [0], 'ICU Discharge': [679.28], 'IV': [(123.08, nan)]},
 141590: {'ICU Admit': [0], 'ICU Discharge': [30.95], 'IV': []},
 141594: {'ICU Admit': [0], 'ICU Discharge': [100.18], 'IV': []},
 141650: {'ICU Admit': [0], 'ICU Discharge': [96.45], 'IV': []},
 141675: {'ICU Admit': [0], 'ICU Discharge': [53.38], 'IV': []},
 141739: {'ICU Admit': [0], 'ICU Discharge': [167.43], 'IV': []},
 141770: {'ICU Admit': [0], 'ICU Discharge': [125.62], 'IV': []},
 141777: {'ICU Admit': [0], 'ICU Discharge': [25.98], 'IV': [(0.17, 0.77)]},
 141806: {'I

In [17]:
len(care_plan_dict.keys())

197396

In [0]:
# Join both dict: keep all icuids from vent event dict. Bring rest of ids from care plan dict

In [0]:
care_plan_dict_copy = care_plan_dict.copy()

In [0]:
for key in vent_event_dict:
  if key in care_plan_dict_copy.keys():
    care_plan_dict_copy.pop(key)

In [39]:
# new dict size:
len(care_plan_dict_copy.keys())

174606

In [0]:
# append vent_event dict to care plan dict:
care_plan_dict_copy.update(vent_event_dict)

In [45]:
len(care_plan_dict_copy.keys())

197465

In [48]:
# check for any duplicates:
sum(pd.Series(list(care_plan_dict_copy.keys())).duplicated())

0

In [0]:
# keep the items with IV start time available (i.e. MV pts) and ICu LOS >= 24h :

In [0]:
IV_LOS24_dict = care_plan_dict_copy.copy()
for i in care_plan_dict_copy.keys():
  admit_offset = care_plan_dict_copy[i]['ICU Admit'][0]
  discharge_offset = care_plan_dict_copy[i]['ICU Discharge'][0]
  LOS_i = discharge_offset - admit_offset

  if (LOS_i>=24.0):   # consider patients with LOS>24hrs
    i_list = care_plan_dict_copy[i]['IV']
    if (len(i_list )==0):  # check for any IV start time. If not pop out that id
      IV_LOS24_dict.pop(i)
    else:
      for ele in i_list:
        if ((ele[0]<0) & (ele[1]<0)) | (np.isnan(ele[0]) & np.isnan(ele[1])): # drop all elements if both IV start and end times < 0 -- occured befer admission time
          print(ele)
          i_list.remove(ele)
      IV_LOS24_dict[i]['IV'] = i_list
      #if (np.isnan(i_list[0][0])): # check if the IV start is nan: if so, pop out
      #IV_LOS24_dict.pop(i)
  else:
    IV_LOS24_dict.pop(i)


In [83]:
# no of ICU stays with MV patients with ICU LOS>=24:
len(IV_LOS24_dict.keys())

43873

In [85]:
# check for any first IV epi start time nan cases:
nan_cnt = 0
for id in IV_LOS24_dict.keys():
  #print(id)
  if np.isnan(IV_LOS24_dict[id]['IV'][0][0]):
    nan_cnt +=1 
print(nan_cnt)


0


In [86]:
IV_LOS24_dict

{141168: {'ICU Admit': [0], 'ICU Discharge': [59.93], 'IV': [(36.0, nan)]},
 141451: {'ICU Admit': [0], 'ICU Discharge': [64.72], 'IV': [(0.87, 16.45)]},
 141515: {'ICU Admit': [0], 'ICU Discharge': [148.03], 'IV': [(0.27, 148.03)]},
 141585: {'ICU Admit': [0], 'ICU Discharge': [679.28], 'IV': [(123.08, nan)]},
 141777: {'ICU Admit': [0], 'ICU Discharge': [25.98], 'IV': [(0.17, 0.77)]},
 141806: {'ICU Admit': [0],
  'ICU Discharge': [647.43],
  'IV': [(0.77, 23.0), (94.57, nan)]},
 141844: {'ICU Admit': [0], 'ICU Discharge': [68.5], 'IV': [(0.88, 18.02)]},
 141853: {'ICU Admit': [0], 'ICU Discharge': [45.9], 'IV': [(1.07, 3.32)]},
 141892: {'ICU Admit': [0], 'ICU Discharge': [25.15], 'IV': [(4.88, 9.02)]},
 142000: {'ICU Admit': [0], 'ICU Discharge': [65.53], 'IV': [(0.47, 18.32)]},
 142099: {'ICU Admit': [0], 'ICU Discharge': [42.12], 'IV': [(22.13, 37.68)]},
 142173: {'ICU Admit': [0],
  'ICU Discharge': [622.22],
  'IV': [(0.38, 604.6), (604.63, nan)]},
 142372: {'ICU Admit': [0], '

## Drop any patient age<15 and keep first ICU stay data for any given patient

In [0]:
# read patients data
df_patients = run_query("""
select patientHealthSystemStayID,patientunitstayid,hospitalAdmitOffset*(-1) as ICUadmit,age
from `physionet-data.eicu_crd.patient`
order by patientHealthSystemStayID, ICUadmit
"""
)

In [103]:
df_patients.head()

,patientHealthSystemStayID,patientunitstayid,ICUadmit,age
0,128919,141168,0,70
1,128927,141178,14,52
2,128927,141179,22,52
3,128941,141194,780,68
4,128943,141197,25,71


In [98]:
df_patients.age.value_counts()

> 89    7081
67      5078
68      4826
72      4804
71      4764
        ... 
9          4
7          3
8          3
3          2
6          1
Name: age, Length: 92, dtype: int64

In [0]:
df_patients.loc[df_patients.age=='> 89','age'] = 90

In [111]:
df_patients.age.value_counts()

90    7081
67    5078
68    4826
72    4804
71    4764
      ... 
5        4
7        3
8        3
3        2
6        1
Name: age, Length: 92, dtype: int64

In [0]:
df_patients.age = pd.to_numeric(df_patients.age,errors='coerce')

In [114]:
df_patients.age.describe()

count    200764.000000
mean         63.079456
std          17.178207
min           0.000000
25%          53.000000
50%          65.000000
75%          76.000000
max          90.000000
Name: age, dtype: float64

In [0]:
# drop pts with age<15:
df_patients_adult = df_patients[df_patients.age>=15.0]

In [117]:
df_patients_adult.age.describe()

count    200634.000000
mean         63.115469
std          17.124778
min          15.000000
25%          53.000000
50%          65.000000
75%          76.000000
max          90.000000
Name: age, dtype: float64

In [122]:
df_patients_adult.iloc[0:20,:]

,patientHealthSystemStayID,patientunitstayid,ICUadmit,age
0,128919,141168,0,70.0
1,128927,141178,14,52.0
2,128927,141179,22,52.0
3,128941,141194,780,68.0
4,128943,141197,25,71.0
5,128943,141196,99,71.0
6,128948,141203,1336,77.0
7,128952,141208,1,25.0
8,128968,141227,1004,82.0
9,128970,141229,4,90.0


In [0]:
# Now pick the first ICU stay from each patient:
df_patients_adult_firastICU = df_patients_adult.groupby('patientHealthSystemStayID')['patientunitstayid'].first()

In [123]:
df_patients_adult_firastICU.values

array([ 141168,  141178,  141194, ..., 3353251, 3353254, 3353263])

In [124]:
len(df_patients_adult_firastICU.values)

166150

In [0]:
# Now pick those ICu ids from our dict:
IV_LOS24_adult_firstICU_dict = IV_LOS24_dict.copy()


In [0]:
for id in IV_LOS24_dict.keys():
  if id not in df_patients_adult_firastICU.values:
    IV_LOS24_adult_firstICU_dict.pop(id)

In [127]:
len(IV_LOS24_adult_firstICU_dict.keys())

38823

In [128]:
IV_LOS24_adult_firstICU_dict

{141168: {'ICU Admit': [0], 'ICU Discharge': [59.93], 'IV': [(36.0, nan)]},
 141451: {'ICU Admit': [0], 'ICU Discharge': [64.72], 'IV': [(0.87, 16.45)]},
 141515: {'ICU Admit': [0], 'ICU Discharge': [148.03], 'IV': [(0.27, 148.03)]},
 141777: {'ICU Admit': [0], 'ICU Discharge': [25.98], 'IV': [(0.17, 0.77)]},
 141806: {'ICU Admit': [0],
  'ICU Discharge': [647.43],
  'IV': [(0.77, 23.0), (94.57, nan)]},
 141844: {'ICU Admit': [0], 'ICU Discharge': [68.5], 'IV': [(0.88, 18.02)]},
 141853: {'ICU Admit': [0], 'ICU Discharge': [45.9], 'IV': [(1.07, 3.32)]},
 142000: {'ICU Admit': [0], 'ICU Discharge': [65.53], 'IV': [(0.47, 18.32)]},
 142099: {'ICU Admit': [0], 'ICU Discharge': [42.12], 'IV': [(22.13, 37.68)]},
 142173: {'ICU Admit': [0],
  'ICU Discharge': [622.22],
  'IV': [(0.38, 604.6), (604.63, nan)]},
 142372: {'ICU Admit': [0], 'ICU Discharge': [40.47], 'IV': [(0.13, 13.07)]},
 142388: {'ICU Admit': [0], 'ICU Discharge': [140.12], 'IV': [(1.23, 99.33)]},
 142723: {'ICU Admit': [0], 

In [141]:
df_patients_adult[df_patients_adult.patientunitstayid==149040]

,patientHealthSystemStayID,patientunitstayid,ICUadmit,age
1212,135026,149040,4334,65.0


In [146]:
df_patients_adult[df_patients_adult.patientHealthSystemStayID==135026]

,patientHealthSystemStayID,patientunitstayid,ICUadmit,age
1212,135026,149040,4334,65.0
1213,135026,149037,38329,65.0
1214,135026,149039,44785,65.0
1215,135026,149038,45370,65.0


In [0]:
f = open("IV_LOS24_adult_firstICU_dict.pkl","wb")
pickle.dump(IV_LOS24_adult_firstICU_dict,f)
f.close()
#files.download('IV_LOS24_adult_firstICU_dict.pkl')